# Embeddings Chatbot: RAG + Datos Tabulares

Este notebook implementa un chatbot que usa embeddings para datos tabulares:
1. **Embeddings** de filas de datos como representaciones semánticas
2. **RAG** (Retrieval Augmented Generation) para encontrar datos relevantes
3. **Semantic Search** para responder preguntas precisas

**Ventajas:**
- Escalable a datasets grandes (10K+ filas)
- Búsqueda semántica inteligente
- Respuestas contextuales específicas
- Manejo eficiente de memoria/tokens

## 1. Configuración del Entorno

In [ ]:
import pandas as pd
import numpy as np
from openai import OpenAI
import os
from dotenv import load_dotenv
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from typing import Dict, List, Any, Tuple
import json
import pickle
from collections import defaultdict

# Cargar variables de entorno
load_dotenv()

# Verificar API key
api_key = os.getenv('OPENAI_API_KEY')
if not api_key:
    print("❌ Error: OPENAI_API_KEY no encontrada en el archivo .env")
    client = None
else:
    try:
        client = OpenAI(api_key=api_key)
        print("✅ Cliente OpenAI inicializado correctamente")
    except Exception as e:
        print(f"❌ Error al inicializar OpenAI: {e}")
        client = None

# Inicializar modelo de embeddings
print("⏳ Cargando modelo de embeddings...")
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
print("✅ Modelo de embeddings cargado")

## 2. Clase Embeddings Chatbot

In [ ]:
class EmbeddingsABTestingChatbot:
    def __init__(self, embedding_model):
        self.data = None
        self.embeddings = None
        self.text_representations = None
        self.embedding_model = embedding_model
        self.client = OpenAI(api_key=os.getenv('OPENAI_API_KEY')) if os.getenv('OPENAI_API_KEY') else None
        self.aggregated_data = None
        self.aggregated_embeddings = None
        self.aggregated_texts = None
    
    def load_data(self, file_path: str):
        """Carga los datos y crea embeddings"""
        if file_path.endswith('.csv'):
            self.data = pd.read_csv(file_path)
        elif file_path.endswith(('.xlsx', '.xls')):
            self.data = pd.read_excel(file_path)
        
        print(f"✅ Datos cargados: {len(self.data)} filas")
        print(f"   Experimentos: {', '.join(self.data['experimento'].unique())}")
        
        # Crear embeddings de filas individuales y agregadas
        self._create_embeddings()
        self._create_aggregated_embeddings()
    
    def _create_row_text(self, row) -> str:
        """Convierte una fila en representación textual descriptiva"""
        return f"""
        Tienda {row['tienda_id']} del experimento {row['experimento']}.
        Ubicada en región {row['region']}, tipo {row['tipo_tienda']}.
        Tiene {row['usuarios']} usuarios, {row['conversiones']} conversiones (tasa {row['conversion_rate']}%).
        Revenue de {row['revenue']:.2f}.
        """.strip()
    
    def _create_embeddings(self):
        """Crea embeddings para cada fila de datos"""
        print("⏳ Creando embeddings de filas individuales...")
        
        # Convertir filas a texto descriptivo
        self.text_representations = []
        for _, row in self.data.iterrows():
            text = self._create_row_text(row)
            self.text_representations.append(text)
        
        # Crear embeddings
        self.embeddings = self.embedding_model.encode(self.text_representations, show_progress_bar=True)
        print(f"✅ {len(self.embeddings)} embeddings creados para filas individuales")
    
    def _create_aggregated_embeddings(self):
        """Crea embeddings para datos agregados - CORREGIDO para evitar doble conteo"""
        print("⏳ Creando embeddings agregados...")
        
        self.aggregated_data = []
        self.aggregated_texts = []
        
        # 1. AGREGACIÓN PRINCIPAL POR EXPERIMENTO (para totales generales)
        exp_summary = self.data.groupby('experimento').agg({
            'usuarios': 'sum',
            'conversiones': 'sum',
            'revenue': 'sum'
        }).round(2)
        exp_summary['conversion_rate'] = (exp_summary['conversiones'] / exp_summary['usuarios'] * 100).round(2)
        exp_summary['revenue_per_user'] = (exp_summary['revenue'] / exp_summary['usuarios']).round(2)
        exp_summary['store_count'] = self.data.groupby('experimento').size()
        
        for exp, metrics in exp_summary.iterrows():
            text = f"""
            RESUMEN TOTAL {exp}: {metrics['usuarios']} usuarios totales en {metrics['store_count']} tiendas.
            Total {metrics['conversiones']} conversiones con tasa {metrics['conversion_rate']}%.
            Revenue total {metrics['revenue']:.2f}, promedio por usuario {metrics['revenue_per_user']:.2f}.
            Este es el TOTAL GENERAL para preguntas de totales de usuarios.
            """.strip()
            self.aggregated_texts.append(text)
            self.aggregated_data.append({
                'type': 'experiment_total',
                'experiment': exp,
                'data': metrics.to_dict(),
                'is_total': True  # Marca para identificar totales
            })
        
        # 2. ANÁLISIS POR REGIÓN (para preguntas específicas de región)
        region_summary = self.data.groupby(['experimento', 'region']).agg({
            'usuarios': 'sum',
            'conversiones': 'sum',
            'revenue': 'sum'
        }).round(2)
        region_summary['conversion_rate'] = (region_summary['conversiones'] / region_summary['usuarios'] * 100).round(2)
        
        for (exp, region), metrics in region_summary.iterrows():
            if metrics['usuarios'] > 0:
                text = f"""
                ANÁLISIS REGIONAL: Experimento {exp} en región {region}.
                {metrics['usuarios']} usuarios, {metrics['conversiones']} conversiones.
                Tasa de conversión {metrics['conversion_rate']}%, revenue {metrics['revenue']:.2f}.
                Esta es información POR REGIÓN, no para totales generales.
                """.strip()
                self.aggregated_texts.append(text)
                self.aggregated_data.append({
                    'type': 'regional_analysis',
                    'experiment': exp,
                    'region': region,
                    'data': metrics.to_dict(),
                    'is_total': False
                })
        
        # 3. ANÁLISIS POR TIPO DE TIENDA (para preguntas específicas de tipo)
        store_summary = self.data.groupby(['experimento', 'tipo_tienda']).agg({
            'usuarios': 'sum',
            'conversiones': 'sum',
            'revenue': 'sum'
        }).round(2)
        store_summary['conversion_rate'] = (store_summary['conversiones'] / store_summary['usuarios'] * 100).round(2)
        
        for (exp, store_type), metrics in store_summary.iterrows():
            if metrics['usuarios'] > 0:
                text = f"""
                ANÁLISIS POR TIPO: Experimento {exp} en tiendas tipo {store_type}.
                {metrics['usuarios']} usuarios, {metrics['conversiones']} conversiones.
                Tasa de conversión {metrics['conversion_rate']}%, revenue {metrics['revenue']:.2f}.
                Esta es información POR TIPO DE TIENDA, no para totales generales.
                """.strip()
                self.aggregated_texts.append(text)
                self.aggregated_data.append({
                    'type': 'store_type_analysis',
                    'experiment': exp,
                    'store_type': store_type,
                    'data': metrics.to_dict(),
                    'is_total': False
                })
        
        # Crear embeddings para datos agregados
        self.aggregated_embeddings = self.embedding_model.encode(self.aggregated_texts, show_progress_bar=True)
        print(f"✅ {len(self.aggregated_embeddings)} embeddings agregados creados")
    
    def find_relevant_data(self, question: str, top_k: int = 10, use_aggregated: bool = True) -> List[Dict]:
        """Encuentra datos relevantes usando búsqueda semántica - CORREGIDO"""
        # Crear embedding de la pregunta
        question_embedding = self.embedding_model.encode([question])
        
        if use_aggregated:
            similarities = cosine_similarity(question_embedding, self.aggregated_embeddings)[0]
            top_indices = similarities.argsort()[-top_k:][::-1]
            
            relevant_data = []
            question_lower = question.lower()
            
            # FILTRADO INTELIGENTE para preguntas de totales
            if any(word in question_lower for word in ['total', 'cuántos', 'suma', 'conjunto']):
                # Para preguntas de totales, SOLO usar datos marcados como 'total'
                for idx in top_indices:
                    item_data = self.aggregated_data[idx]
                    if item_data.get('is_total', False):  # Solo totales generales
                        relevant_data.append({
                            'similarity': similarities[idx],
                            'text': self.aggregated_texts[idx],
                            'data': item_data
                        })
                        if len(relevant_data) >= 3:  # Limitar a pocos resultados para totales
                            break
            else:
                # Para otras preguntas, usar todos los datos relevantes
                for idx in top_indices:
                    relevant_data.append({
                        'similarity': similarities[idx],
                        'text': self.aggregated_texts[idx],
                        'data': self.aggregated_data[idx]
                    })
        else:
            # Usar datos de filas individuales
            similarities = cosine_similarity(question_embedding, self.embeddings)[0]
            top_indices = similarities.argsort()[-top_k:][::-1]
            
            relevant_data = []
            for idx in top_indices:
                row_data = self.data.iloc[idx].to_dict()
                relevant_data.append({
                    'similarity': similarities[idx],
                    'text': self.text_representations[idx],
                    'data': {'type': 'individual_store', 'store_data': row_data}
                })
        
        return relevant_data
    
    def determine_search_strategy(self, question: str) -> bool:
        """Determina si usar datos agregados o individuales"""
        # Palabras que indican consultas agregadas
        aggregated_keywords = [
            'total', 'cuántos', 'suma', 'comparar', 'mejor', 'peor',
            'promedio', 'general', 'resumen', 'experimento',
            'todos', 'todas', 'conjunto'
        ]
        
        # Palabras que indican consultas individuales
        individual_keywords = [
            'tienda', 'específica', 'particular', 'individual',
            'detalle', 'ejemplo'
        ]
        
        question_lower = question.lower()
        
        aggregated_score = sum(1 for keyword in aggregated_keywords if keyword in question_lower)
        individual_score = sum(1 for keyword in individual_keywords if keyword in question_lower)
        
        # Por defecto usar agregados para consultas generales
        return aggregated_score >= individual_score
    
    def generate_context(self, relevant_data: List[Dict]) -> str:
        """Genera contexto optimizado para el LLM - MEJORADO"""
        context_parts = ["DATOS RELEVANTES ENCONTRADOS:"]
        
        # Verificar si hay datos de totales vs análisis detallados
        total_data = [item for item in relevant_data if item['data'].get('is_total', False)]
        analysis_data = [item for item in relevant_data if not item['data'].get('is_total', False)]
        
        if total_data:
            context_parts.append("\n🎯 DATOS TOTALES (usar estos para sumas generales):")
            for i, item in enumerate(total_data, 1):
                context_parts.append(f"[{i}] {item['text']}")
                if item['data']['type'] == 'experiment_total':
                    data_summary = item['data']['data']
                    context_parts.append(f"   TOTAL EXACTO: {data_summary.get('usuarios', 'N/A')} usuarios")
        
        if analysis_data and not any(word in relevant_data[0]['text'].lower() for word in ['total', 'resumen total']):
            context_parts.append("\n📊 ANÁLISIS DETALLADOS (NO sumar estos):")
            for i, item in enumerate(analysis_data[:3], 1):  # Limitar análisis detallados
                context_parts.append(f"[{i}] {item['text'][:100]}...")
        
        return "\n".join(context_parts)
    
    def chat(self, question: str) -> str:
        """Función principal del chatbot con embeddings"""
        if self.data is None:
            return "❌ Primero carga los datos usando load_data()"
        
        if self.client is None:
            return "❌ Cliente OpenAI no disponible"
        
        # 1. Determinar estrategia de búsqueda
        use_aggregated = self.determine_search_strategy(question)
        search_type = "agregados" if use_aggregated else "individuales"
        print(f"🔍 Usando datos {search_type}")
        
        # 2. Encontrar datos relevantes
        relevant_data = self.find_relevant_data(question, top_k=8, use_aggregated=use_aggregated)
        print(f"📊 {len(relevant_data)} elementos relevantes encontrados")
        
        # 3. Generar contexto
        context = self.generate_context(relevant_data)
        
        # 4. Generar respuesta
        system_prompt = """
        Eres un analista experto en A/B testing. Respondes preguntas basándote en datos relevantes encontrados mediante búsqueda semántica.
        
        IMPORTANTE PARA EVITAR DOBLE CONTEO:
        - Si ves "DATOS TOTALES", usa EXACTAMENTE esos números para responder sobre totales
        - NO sumes datos de diferentes categorías (región + tipo de tienda)
        - Los datos regionales y por tipo son para análisis específicos, NO para totales generales
        - Si la pregunta es sobre total de usuarios, busca el número marcado como "TOTAL EXACTO"
        - Nunca sumes usuarios por región Y por tipo de tienda (son los mismos usuarios)
        
        Para cálculos:
        - Usa los datos marcados como "TOTAL" para preguntas generales
        - Usa análisis detallados solo para comparaciones específicas
        - Si hay inconsistencia, prioriza los datos marcados como totales oficiales
        """
        
        user_prompt = f"""
        PREGUNTA: {question}
        
        CONTEXTO:
        {context}
        
        Responde la pregunta basándote en los datos relevantes. Para totales de usuarios, usa SOLO los datos marcados como totales, NO sumes categorías.
        """
        
        try:
            response = self.client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}
                ],
                max_tokens=800,
                temperature=0.2
            )
            return response.choices[0].message.content
        except Exception as e:
            return f"❌ Error al generar respuesta: {e}"
    
    def save_embeddings(self, file_path: str):
        """Guarda embeddings para uso futuro"""
        embedding_data = {
            'embeddings': self.embeddings,
            'text_representations': self.text_representations,
            'aggregated_embeddings': self.aggregated_embeddings,
            'aggregated_texts': self.aggregated_texts,
            'aggregated_data': self.aggregated_data
        }
        with open(file_path, 'wb') as f:
            pickle.dump(embedding_data, f)
        print(f"✅ Embeddings guardados en {file_path}")
    
    def load_embeddings(self, file_path: str):
        """Carga embeddings previamente guardados"""
        try:
            with open(file_path, 'rb') as f:
                embedding_data = pickle.load(f)
            
            self.embeddings = embedding_data['embeddings']
            self.text_representations = embedding_data['text_representations']
            self.aggregated_embeddings = embedding_data['aggregated_embeddings']
            self.aggregated_texts = embedding_data['aggregated_texts']
            self.aggregated_data = embedding_data['aggregated_data']
            
            print(f"✅ Embeddings cargados desde {file_path}")
            return True
        except Exception as e:
            print(f"❌ Error cargando embeddings: {e}")
            return False
    
    def show_search_preview(self, question: str, top_k: int = 5):
        """Muestra vista previa de búsqueda semántica"""
        use_aggregated = self.determine_search_strategy(question)
        relevant_data = self.find_relevant_data(question, top_k=top_k, use_aggregated=use_aggregated)
        
        print(f"🔍 Búsqueda para: '{question}'")
        print(f"📊 Estrategia: {'Agregados' if use_aggregated else 'Individuales'}")
        print(f"\n🎯 Top {len(relevant_data)} resultados más relevantes:")
        
        for i, item in enumerate(relevant_data, 1):
            is_total = item['data'].get('is_total', False)
            total_indicator = " [TOTAL]" if is_total else ""
            print(f"\n[{i}] Relevancia: {item['similarity']:.3f}{total_indicator}")
            print(f"    {item['text'][:100]}...")
            if is_total and 'data' in item['data']:
                usuarios = item['data']['data'].get('usuarios', 'N/A')
                print(f"    >>> TOTAL EXACTO: {usuarios} usuarios")

## 3. Crear Datos de Prueba

In [ ]:
# Usar los mismos datos base
np.random.seed(42)

# Datos detallados por tienda
tiendas_detalle = []
experimentos = ['Control', 'Experimento_A', 'Experimento_B', 'Experimento_C']
base_conversion_rates = [8.5, 9.4, 10.2, 11.9]

for exp, base_conversion in zip(experimentos, base_conversion_rates):
    for i in range(50):  # 50 tiendas por experimento
        tiendas_detalle.append({
            'experimento': exp,
            'tienda_id': f'T_{exp}_{i+1:03d}',
            'region': np.random.choice(['Norte', 'Sur', 'Este', 'Oeste']),
            'tipo_tienda': np.random.choice(['Mall', 'Street', 'Outlet']),
            'usuarios': np.random.randint(150, 300),
            'conversiones': None,
            'revenue': None,
            'conversion_rate': np.random.normal(base_conversion, 2.5)
        })

tiendas_df = pd.DataFrame(tiendas_detalle)
tiendas_df['conversiones'] = (tiendas_df['usuarios'] * tiendas_df['conversion_rate'] / 100).round().astype(int)
tiendas_df['revenue'] = tiendas_df['conversiones'] * np.random.uniform(15, 25, len(tiendas_df))
tiendas_df['conversion_rate'] = tiendas_df['conversion_rate'].round(2)

# Guardar archivo
tiendas_df.to_csv('embeddings_tiendas_detalle.csv', index=False)

print("✅ Archivo embeddings_tiendas_detalle.csv creado")
print(f"📊 {len(tiendas_df)} filas, {len(experimentos)} experimentos")

## 4. Inicializar Chatbot con Embeddings

In [ ]:
# Crear chatbot con embeddings
embeddings_bot = EmbeddingsABTestingChatbot(embedding_model)

# Cargar datos (esto creará automáticamente los embeddings)
embeddings_bot.load_data('embeddings_tiendas_detalle.csv')

# Opcional: Guardar embeddings para uso futuro
embeddings_bot.save_embeddings('ab_testing_embeddings.pkl')

## 5. Ejemplos de Uso

In [ ]:
# Ejemplo 1: Total usuarios Control
pregunta_1 = "¿Cuántos usuarios en total tienen las tiendas control?"
print(f"🤔 Pregunta: {pregunta_1}")
print(f"💬 Respuesta: {embeddings_bot.chat(pregunta_1)}")
print("\n" + "="*80 + "\n")

In [ ]:
# Ejemplo 2: Vista previa de búsqueda
embeddings_bot.show_search_preview("¿Cuántos usuarios en total tienen las tiendas control?")
print("\n" + "="*80 + "\n")

In [ ]:
# Ejemplo 3: Mejor experimento
pregunta_3 = "¿Cuál es el mejor experimento?"
print(f"🤔 Pregunta: {pregunta_3}")
print(f"💬 Respuesta: {embeddings_bot.chat(pregunta_3)}")
print("\n" + "="*80 + "\n")

In [ ]:
# Ejemplo 4: Análisis regional
pregunta_4 = "¿Cómo se comporta el experimento Control en la región Norte?"
print(f"🤔 Pregunta: {pregunta_4}")
embeddings_bot.show_search_preview(pregunta_4)
print(f"\n💬 Respuesta: {embeddings_bot.chat(pregunta_4)}")
print("\n" + "="*80 + "\n")

In [ ]:
# Ejemplo 5: Comparación de tipos de tienda
pregunta_5 = "¿Qué tipo de tienda funciona mejor en el Experimento_B?"
print(f"🤔 Pregunta: {pregunta_5}")
print(f"💬 Respuesta: {embeddings_bot.chat(pregunta_5)}")
print("\n" + "="*80 + "\n")

## 6. Comparación de Estrategias

In [ ]:
# Comparar búsqueda agregada vs individual
test_questions = [
    "¿Cuántos usuarios totales tiene Control?",
    "Dame detalles de la tienda T_Control_001",
    "¿Cuál es el mejor experimento?",
    "¿Qué tienda específica tiene más conversiones?",
    "Compara todos los experimentos"
]

print("🔬 Análisis de estrategias de búsqueda:")
for question in test_questions:
    use_aggregated = embeddings_bot.determine_search_strategy(question)
    strategy = "📊 Agregados" if use_aggregated else "🏪 Individuales"
    print(f"   {strategy}: '{question}'")

## 7. Chat Interactivo

In [ ]:
def chat_interactivo_embeddings():
    """Chat interactivo con el chatbot de embeddings"""
    print("🤖 Embeddings Chatbot iniciado!")
    print("💡 Escribe 'salir' para terminar")
    print("🔍 Escribe 'buscar: tu pregunta' para ver vista previa de búsqueda\n")
    
    while True:
        pregunta = input("🤔 Tu pregunta: ")
        
        if pregunta.lower() in ['salir', 'exit', 'quit']:
            print("👋 ¡Hasta luego!")
            break
        
        if pregunta.lower().startswith('buscar:'):
            search_query = pregunta[7:].strip()
            embeddings_bot.show_search_preview(search_query)
            continue
            
        if pregunta.strip() == "":
            continue
            
        print(f"\n💬 Respuesta: {embeddings_bot.chat(pregunta)}")
        print("\n" + "-"*50 + "\n")

# Descomentar para usar el chat interactivo
# chat_interactivo_embeddings()

## 8. Análisis de Performance

In [ ]:
# Análisis de similitudes para diferentes preguntas
test_questions_analysis = [
    "¿Cuántos usuarios control?",
    "Total de usuarios en Control",
    "Usuarios grupo control",
    "¿Cuál es el mejor experimento?",
    "Experimento más exitoso",
    "¿Qué funciona mejor?"
]

print("📈 Análisis de similitud semántica:")
for question in test_questions_analysis:
    relevant_data = embeddings_bot.find_relevant_data(question, top_k=3)
    print(f"\n🔍 '{question}'")
    for i, item in enumerate(relevant_data[:2], 1):
        print(f"   [{i}] {item['similarity']:.3f} - {item['text'][:60]}...")

## 9. Verificación de Precisión

In [ ]:
# Verificar que los embeddings devuelven datos correctos
print("🔍 Verificación de precisión - Total usuarios Control:")

# Cálculo manual
manual_control = tiendas_df[tiendas_df['experimento'] == 'Control']['usuarios'].sum()
print(f"📊 Manual: {manual_control} usuarios")

# Buscar con embeddings
relevant_data = embeddings_bot.find_relevant_data("total usuarios control", top_k=5)
print(f"\n🎯 Datos más relevantes encontrados:")
for item in relevant_data[:3]:
    if item['data']['type'] == 'experiment_summary' and item['data']['experiment'] == 'Control':
        usuarios_embeddings = item['data']['data']['usuarios']
        print(f"🤖 Embeddings: {usuarios_embeddings} usuarios")
        print(f"✅ Coinciden: {manual_control == usuarios_embeddings}")
        break

print(f"\n📋 Top 3 resultados por relevancia:")
for i, item in enumerate(relevant_data[:3], 1):
    print(f"[{i}] Sim: {item['similarity']:.3f} - {item['text'][:80]}...")